# get_frequently_mutated Final 

In [1]:
def get_frequently_mutated(cancer_type, cutoff=.1):  
    """take cancer type to import cptac and find the frequently mutated genes in total tumors compared to the cutoff.
        
        Parameters:
        cancer_type (string): type of  cancer 
        cutoff (float): used as comparison to determine status of gene mutation frequency
        
        Returns:
        freq_mutated_df (pd.DataFrame): DataFrame of frequently mutated genes passing the cutoff
            and percent mutated (mutated genes / total tumors), percent Truncated, percent Missence
        
        There are many types of mutations catagorized into the columns 'Truncated' and 'Missence'. 
        The Truncated column includes: Frame_Shift_Del, Frame_Shift_Ins, Splice_Site, Nonsense_Mutation, Nonstop_Mutation
        The Missence column includes: In_Frame_Del, In_Frame_Ins, Missense_Mutation
        These columns count multiple mutations of one gene in the same sample, so percentages in the last two columns may 
        exceed the total percent mutated (which only counts if the gene was mutated once)
            """ 
    
    # import CPTAC and pandas
    
    import pandas as pd
    colon = False
    if cancer_type == "endometrial" or cancer_type == "Endometrial":
        import cptac.endometrial as cptac
        
    elif cancer_type == "colon" or cancer_type == "Colon":
        import cptac.colon as cptac
        colon = True
        
    elif cancer_type == "ovarian" or cancer_type == "Ovarian":
        import cptac.ovarian as cptac
    
    else:
        str_cancer_options = '\n' + 'Options: endometrial, ovarian, colon'
        print("Please enter a valid cancer type.", str_cancer_options)
        return 0
    
    # get data frames
    somatic_mutations = cptac.get_mutations()
    proteomics = cptac.get_proteomics()
    sample_status_map = cptac.get_sample_status_map()
    merged_mutations = somatic_mutations.join(sample_status_map, how="left") 
    
    # standardize mutation names 
    if colon == True:
        mutation_equivalents = {'frameshift substitution': 'Frame_Shift_Del' , 'frameshift deletion': 'Frame_Shift_Del', 
            'frameshift insertion': 'Frame_Shift_Ins', 'stopgain': 'Nonsense_Mutation ', 'stoploss':'Nonstop_Mutation',
            'nonsynonymous SNV': 'Missense_Mutation','nonframeshift insertion': 'In_Frame_Ins',
            'nonframeshift deletion': 'In_Frame_Del', 'nonframeshift substitution': 'Missense_Mutation'}
        merged_mutations = merged_mutations.replace(to_replace = mutation_equivalents)
        
    # get list of unique genes
    unique_genes = somatic_mutations['Gene'].unique()
    
    # get total tumors/patients
    sample_status_series = sample_status_map.value_counts()
    total_tumor_patients = sample_status_series[0]
        
    # find frequently mutated genes and their percent mutated. Create lists for frequently mutated genes and percentage.
    freq_mut = []
    total_percent_mutated = []
    for gene in unique_genes:
        gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
        gene_mutated = gene_mutated.index.unique()
        num_gene_mutated = len(gene_mutated)
        percentage = (num_gene_mutated / total_tumor_patients)
        if percentage > cutoff:
            freq_mut.append(gene)
            total_percent_mutated.append(percentage)
    
    # find truncated percentage
    truncated = []
    missence = []
    sample_overlap = []
    
    for gene in freq_mut:
        gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
        
        # trunc
        truncated_df = gene_mutated.loc[(gene_mutated['Mutation'] != 'In_Frame_Del') & 
            (gene_mutated['Mutation'] != 'In_Frame_Ins') & (gene_mutated['Mutation'] != 'Missense_Mutation')] 
        samples_trunc = truncated_df.index.unique()
        num_trunc_mut = len(samples_trunc)
        fraction_trunc = (num_trunc_mut / total_tumor_patients)
        truncated.append(fraction_trunc)
        
        #miss
        missence_mutations = gene_mutated.loc[(gene_mutated['Mutation'] == 'In_Frame_Ins') 
                | (gene_mutated['Mutation'] == 'In_Frame_Del') | (gene_mutated['Mutation'] == 'Missense_Mutation')]
        samples_miss = missence_mutations.index.unique()
        num_miss_mut = len(samples_miss)
        
        #count overlap in truncated
        num_overlap = 0
        non_overlap_samples_miss = []
        for sample in samples_miss:
            if sample not in samples_trunc:
                non_overlap_samples_miss.append(sample)
            else:
                num_overlap += 1
        
        num_non_overlap_samples = len(non_overlap_samples_miss)
        fraction_miss = (num_non_overlap_samples / total_tumor_patients)
        missence.append(fraction_miss)
        
    # create dataframe
    merged_lists = list(zip(freq_mut, total_percent_mutated, truncated, missence))
    freq_mutated_df = pd.DataFrame(merged_lists, columns =['Gene', 'Fraction_Mutated', 'Fraction_Truncation', 'Fraction_Missence'])
                   
    return freq_mutated_df

In [2]:
endo_freq_mutated_df = get_frequently_mutated("endometrial", .25)

Welcome to the cptac data service package. Available datasets may be
viewed using cptac.list_data(). In order to access a specific data
set, import a cptac subfolder using either 'import cptac.dataset' or
'from cptac import dataset'.
******
Version: 0.4.1
******
You have loaded the cptac endometrial dataset. To view available
dataframes, use cptac.endometrial.list_data(). To view available
functions for accessing and manipulating the dataframes, use
cptac.endometrial.list_api().
endometrial data version: 2.1

Loading Dictionary...
Loading cptac endometrial data:
Loading acetylproteomics data...
Loading clinical data...
Loading CNA data...
Loading miRNA data...
Loading phosphoproteomics_gene data...
Loading phosphoproteomics_site data...
Loading proteomics data...
Loading somatic data...
Loading somatic_binary data...
Loading transcriptomics_circular data...
Loading transcriptomics_linear data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available

In [3]:
endo_freq_mutated_df

,Gene,Fraction_Mutated,Fraction_Truncation,Fraction_Missence
0,ARID1A,0.452632,0.400000,0.052632
1,PIK3CA,0.494737,0.010526,0.484211
2,PTEN,0.789474,0.568421,0.221053
3,CTCF,0.284211,0.242105,0.042105
4,KRAS,0.326316,0.000000,0.326316
5,PIK3R1,0.389474,0.189474,0.200000
6,CTNNB1,0.305263,0.000000,0.305263


Test Colon

In [91]:
colon_freq_mutated_df = get_frequently_mutated("colon", .25)

You have loaded the cptac colon dataset. To view available dataframes,
use cptac.colon.list_data(). To view available functions for accessing
and manipulating the dataframes, use cptac.colon.list_api().
colon data version: Most recent release

Loading cptac colon data:
Loading clinical data...
Loading miRNA data...
Loading mutation data...
Loading mutation_binary data...
Loading phosphoproteomics_normal data...
Loading phosphoproteomics_tumor data...
Loading proteomics_normal data...
Loading proteomics_tumor data...
Loading transcriptomics data...
KRAS num_gene_mutated 35
KRAS num_trunc_mut 0
KRAS num miss mut 35
KRAS num overlap 0
KRAS num_non_overlap_samples 35
KRAS check: 0 + 35 = 35 



In [92]:
colon_freq_mutated_df

,Gene,Fraction_Mutated,Fraction_Truncation,Fraction_Missence
0,KRAS,0.318182,0.0,0.318182


Test Ovarian, cutoff = .1 (default)

In [93]:
ovarian_freq_mutated_df = get_frequently_mutated("ovarian")

You have loaded the cptac ovarian dataset. To view available
dataframes, use cptac.ovarian.list_data(). To view available functions
for accessing and manipulating the dataframes, use
cptac.ovarian.list_api().
ovarian data version: Most recent release

Loading cptac ovarian data:
Loading clinical data...
Loading cnv data...
Loading phosphoproteomics data...
Loading proteomics data...
Loading somatic_38 data...
Loading transcriptomics data...
Loading treatment data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until June 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter embargo() to open the webpage for more details.


In [94]:
ovarian_freq_mutated_df

,Gene,Fraction_Mutated,Fraction_Truncation,Fraction_Missence


Debug 

In [17]:
#find frequently mutated - check Ovarian PTEN percentage = .9277 
somatic = c.get_mutations()
prot = c.get_proteomics()
gene = 'PTEN'
omics_mutations = c.append_mutations_to_omics(mutation_genes=gene, omics_df=prot, omics_genes=gene)
unique_genes = ['TP53','PTEN']
#somatic['Gene'].unique()
gene_mutated = somatic.loc[somatic['Gene'] == 'PTEN']
gene_and_freq_d = {}
tumors = omics_mutations.loc[omics_mutations['Sample_Status'] == 'Tumor']
total_tumor_patients = len(tumors)
print('total_tumor_patients:', total_tumor_patients)
cutoff = .25
for gene in unique_genes:
    gene_mutated = somatic.loc[somatic['Gene'] == gene].index
    num_gene_mutated = len(gene_mutated.unique())
    print('num_gene_mutated', gene, ':', num_gene_mutated)
   
    percentage = (num_gene_mutated / total_tumor_patients)
    if percentage > cutoff:
        gene_and_freq_d[gene] = percentage

total_tumor_patients: 83
num_gene_mutated TP53 : 77
num_gene_mutated PTEN : 4


In [19]:
gene_and_freq_d

{'TP53': 0.927710843373494}

In [20]:
gene = 'TP53'
gene_mutated = somatic.loc[somatic['Gene'] == gene].index
num_gene_mutated = float(len(gene_mutated.unique()))
print('gene_mutated: ', num_gene_mutated, '/', total_tumor_patients, 'total_tumor_patients')
percentage = (num_gene_mutated / total_tumor_patients)
print('percent:',percentage)
gene_mutated.unique()

gene_mutated:  77.0 / 83 total_tumor_patients
percent: 0.927710843373494


Index(['S002', 'S006', 'S007', 'S009', 'S011', 'S013', 'S014', 'S015', 'S020',
       'S021', 'S022', 'S024', 'S025', 'S026', 'S027', 'S029', 'S031', 'S033',
       'S035', 'S036', 'S037', 'S038', 'S039', 'S040', 'S041', 'S042', 'S043',
       'S044', 'S045', 'S047', 'S048', 'S049', 'S050', 'S051', 'S052', 'S053',
       'S054', 'S055', 'S057', 'S058', 'S059', 'S060', 'S061', 'S062', 'S063',
       'S064', 'S065', 'S066', 'S067', 'S068', 'S069', 'S073', 'S074', 'S075',
       'S076', 'S082', 'S083', 'S086', 'S087', 'S089', 'S090', 'S093', 'S095',
       'S096', 'S097', 'S098', 'S100', 'S103', 'S106', 'S107', 'S108', 'S109',
       'S111', 'S112', 'S113', 'S115', 'S116'],
      dtype='object', name='Sample_ID')